In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gensim==3.4.0
!pip install smart_open==1.9.0

# **Pretraining**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import re

tweets_sg_100 = '/kaggle/input/tweets-sg-100/tweets_sg_100'
sg_model = gensim.models.Word2Vec.load(tweets_sg_100)
df = pd.read_csv("/kaggle/input/tweets-pipeline/tweets_0.csv")
df.head()

In [ ]:
russia = ['روسي','روس','روسيه','لروسي','والروس','ياروسي','روسی','موسك'] # 'روسيا'
belarus = ['يلاروسي','يلاروس','بيلاروس','بيلاروسيه','لبيلاروسي','والبيلاروس'] # 'بيلاروسيا'
ukr = ['اوكران','واوكراني','اوكر','لاوكراني','ناتو'] # 'اوكرانيا'
russian_hashtags = ['جيشالروس','قواتالروسيه','روسياليوم','روسياتنتصر','خارجيهالروسيه','عمليهالعسكريهالروسيه'] # روسيا
ukr_russian_hashtags = ['حربالروسيهالاوكرانيه','روسياوكراني','اوكرانياروسي','غزوالروسيلاوكرانيا','روسياواوكراني'] # 'حرب'
ukr_hashtags = ['روسياتغزواوكراني','روسياتكذب','روسيالص','روسياوالص','روسيامجرم','مجدلاوكرانيا','مخابراتالاوكرانيه'] # 'اوكرانيا'
zelensky = ['زيلينسك','زلينسك'] # 'اوباما'
putin = ['بوتن','وتن','پوتن','وتين','بوتين','پوتين'] #'بوتين'
war = ['حربالعالميه', 'حربالعالميهالثالثه','عسكريه'] # 'حرب'
egypt = ['مصر','مصريه','لمصر'] # 'عربي'
us = ['وامريك','الولاياتالمتحده','الولاياتالمتحده','الولاياتالمتحدهالامريكيه','البريطانيا','والولايا'] # 'امريكا'
gb = ['ريطاني'] # "بريطانيا"

russia.extend(russian_hashtags)
ukr.extend(ukr_hashtags)
war.extend(ukr_russian_hashtags)

added_words = []
added_words.extend(russia)
added_words.extend(ukr)
added_words.extend(war)
added_words.extend(putin)
added_words.extend(zelensky)
added_words.extend(egypt)
added_words.extend(us)
added_words.extend(gb)
added_words = set(added_words)
russia = set(russia)
ukr = set(ukr)
war = set(war)
putin = set(putin)
zelensky = set(zelensky)
egypt = set(egypt)
us = set(us)
gb = set(gb)

# add words that donot exist to wv
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
for word in russia:
    sg_model.wv.add_vector(word,sg_model.wv['روسيا'])
for word in ukr:
    sg_model.wv.add_vector(word,sg_model.wv['اوكرانيا'])
for word in war:
    sg_model.wv.add_vector(word,sg_model.wv['حرب'])
for word in putin:
    sg_model.wv.add_vector(word,sg_model.wv['بوتين'])
for word in zelensky:
    sg_model.wv.add_vector(word,sg_model.wv['اوباما'])
for word in belarus:
    sg_model.wv.add_vector(word,sg_model.wv['بيلاروسيا'])
for word in egypt:
    sg_model.wv.add_vector(word,sg_model.wv['قطر'])   
for word in us:
    sg_model.wv.add_vector(word,sg_model.wv['امريكا']) 
for word in gb:
    sg_model.wv.add_vector(word,sg_model.wv["بريطانيا"])

In [ ]:
from tensorflow.keras.utils import to_categorical
def load_pretraining(file_path):
    df = pd.read_csv(file_path)
    sentences = df["tweet_text"].astype(str)
    ex_length = len(df)
    X1 = tf.zeros([ex_length,64,100],dtype=tf.float32)
    X2 = tf.zeros([ex_length,64,100],dtype=tf.float32)
    for ex in range(ex_length):
        words = sentences[ex].split()
        position = 0
        for word in words:
            if word in sg_model.wv:
                X1[ex,position,:] = sg_model.wv[word]
                X2[ex,position+1,:].assign(sg_model.wv[word])
    print(X1.shape,X2.shape)     
    return (X1,X2),X1

In [ ]:
file_paths = []
for i in range(14):
    file_paths.append("/kaggle/input/tweets-pipeline/tweets_"+str(i)+".csv")

In [ ]:
def tweet_generator_pretraining():
    for file_path in file_paths:
        X,y = load_pretraining(file_path)
        for tweet,label in zip(X,y):
            yield tweet,label

In [ ]:
_ = load_pretraining("/kaggle/input/tweets-pipeline/tweets_9.csv")

In [ ]:
# Create a TensorFlow Dataset
import tensorflow as tf
pretraining_dataset = tf.data.Dataset.from_generator(tweet_generator_pretraining,output_signature=
                                                     ((tf.TensorSpec(shape=(64, 100), dtype=tf.float32),
                                                     tf.TensorSpec(shape=(64, 100), dtype=tf.float32)), 
                                                      tf.TensorSpec(shape=(64,100), dtype=tf.float32)))                                                                           
batch_size = 8
dataset = pretraining_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for (encoder_input, decoder_input), target_output in pretraining_dataset.take(1):
    print('Encoder input shape:', encoder_input.shape)
    print('Decoder input shape:', decoder_input.shape)
    print('Target output shape:', target_output.shape)

In [ ]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Dense, Flatten

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 100),name='e_inp') 
decoder_inputs = Input(shape=(None, 100),name='d_inp')
encoder = Bidirectional(LSTM(64, return_state=True),name='enc')
encoder_outputs, hf, hb, cf, cb = encoder(encoder_inputs)
encoder_states = [hf, hb, cf, cb]
decoder_lstm = Bidirectional(LSTM(64, return_sequences=True, return_state=True),name='dec')
decoder_outputs, dhf, dhb, dcf, dcb = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(100,name="dense_1")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
model.load_weights("/kaggle/input/best_weights/keras/best_best/2/pretrained_weights.weights.h5")

In [ ]:
from keras.losses import CosineSimilarity, MeanSquaredError
cs_loss = CosineSimilarity(axis=-1, reduction="sum_over_batch_size", name="cosine_similarity")
model.compile(optimizer='adam',loss=cs_loss ,metrics=['acc'])
model.fit(pretrainig_dataset,
          epochs=10)


In [ ]:
model.save_weights("pre_model_bilstm.weights.h5")

# **Partiality** 

In [ ]:
encoder = model.get_layer('enc')
config = encoder.get_config()
config['layer']['config']['return_state'] = False
config['layer']['config']['return_sequences'] = True

encoder_layer_new = Bidirectional(LSTM.from_config(config['layer']['config']), name='enc')

# Get the weights of the original encoder layer and set them to the new layer
encoder_layer_new.build((None,None,100))
encoder_layer_new.set_weights(encoder.get_weights())       

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

# Define the model
model = Sequential()
# Add Bidirectional LSTM layers
model.add(encoder_layer_new)
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))

# Add Dense layer
model.add(Dense(units=20, activation='relu'))

# Add Softmax layer
model.add(Dense(units=3, activation='softmax'))

loss= CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="sum_over_batch_size",
    name="categorical_crossentropy",
)
# Compile the model
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
def load_and_preprocess_tweets(file_path):
    df = pd.read_csv(file_path)
    sentences = df["tweet_text"].astype(str)
    y= to_categorical(df['label'])
    ex_length = len(df)
    X = np.zeros([ex_length,64,100])
    for ex in range(ex_length):
        words = sentences[ex].split()
        position = 0
        for word in words:
            if word in sg_model.wv:
                X[ex,position,:] = sg_model.wv[word]
            position += 1
       
    return X,y

In [ ]:
def tweet_generator():
    for file_path in file_paths:
        X, y = load_and_preprocess_tweets(file_path)
        for tweet, label in zip(X, y):
            yield tweet, label

In [ ]:
# Create a TensorFlow Dataset
import tensorflow as tf
dataset = tf.data.Dataset.from_generator(tweet_generator,output_signature=(tf.TensorSpec(shape=(None,100), dtype=tf.float32), tf.TensorSpec(shape=(3,), dtype=tf.float32)))                                                                           
batch_size = 32
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
X_val, y_val = load_and_preprocess_tweets("/kaggle/input/tweets-pipeline/tweets_12.csv")
X_test, y_test = load_and_preprocess_tweets("/kaggle/input/tweets-pipeline/tweets_13.csv")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint filepath with '.keras' extension
checkpoint_filepath = 'best_weights.keras'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                      save_best_only=True,
                                      monitor='val_accuracy',
                                      mode='max',  # 'max' for accuracy
                                      verbose=1)

# Train the model with ModelCheckpoint callback
history = model.fit(dataset,
                    epochs=1,
                    callbacks=[checkpoint_callback],shuffle = True,validation_data=(X_val,y_val))


In [ ]:
model.load_weights("/kaggle/input/best_weights/keras/best_best/1/best_weights-9.keras")

In [ ]:
loss,accuracy = model.evaluate(X_test,y_test)

print(f"Loss :{loss:.4f}")
print(f"Accuracy :{accuracy:.4f}")

In [ ]:
model.load_weights("/kaggle/input/normal22/keras/v1/1/model.weights.h5")

In [ ]:
# model 4 is the best model
# model.weights.h5

In [ ]:
# self training
# pick tweet 0
# and run prediction on it 
# and see the least confident predictions 
# get their indices and from indices get their 
# label them manually
# same for x_test
# 